<a href="https://colab.research.google.com/github/ngtaylor/EE460J-Final-Project/blob/main/EE460J_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using BERT


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

     |████████████████████████████████| 4.9 MB 8.3 MB/s 


In [ ]:
!pip install -q tf-models-official

     |████████████████████████████████| 1.8 MB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 79.0 MB/s 
     |████████████████████████████████| 90 kB 13.4 MB/s 
     |████████████████████████████████| 99 kB 12.5 MB/s 
     |████████████████████████████████| 47.6 MB 1.2 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 213 kB 68.7 MB/s 
     |████████████████████████████████| 1.1 MB 69.6 MB/s 
     |████████████████████████████████| 596 kB 72.1 MB/s 
     |████████████████████████████████| 352 kB 79.0 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 
     |████████████████████████████████| 3.3 MB 53.1 MB/s 
     |████████████████████████████████| 61 kB 608 kB/s 


In [ ]:
import os
import shutil


import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

from datetime import datetime

import matplotlib.pyplot as plt

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
!wget https://github.com/ngtaylor/EE460J-Final-Project/raw/main/data/training.1600000.processed.noemoticon.csv.zip

--2021-12-06 17:11:44--  https://github.com/ngtaylor/EE460J-Final-Project/raw/main/data/training.1600000.processed.noemoticon.csv.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ngtaylor/EE460J-Final-Project/main/data/training.1600000.processed.noemoticon.csv.zip [following]
--2021-12-06 17:11:44--  https://raw.githubusercontent.com/ngtaylor/EE460J-Final-Project/main/data/training.1600000.processed.noemoticon.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84855679 (81M) [application/zip]
Saving to: ‘training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[=======

In [ ]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv.zip', encoding='latin-1', names=["target", "id", "date", "flag", "user", "text"])

In [ ]:
df.head()


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.shape

(1600000, 6)

In [ ]:
#Change positive target value from 4 to 1 and drop flag column since it is useless
df["target"] = df["target"].replace({4:1})
df = df.drop('flag', axis=1)
df.head()

,target,id,date,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#remove URL
df['text'] = df['text'].str.replace(r'http(\S)+', r'')
df['text'] = df['text'].str.replace(r'http ...', r'')
df['text'] = df['text'].str.replace(r'http', r'')
df[df['text'].str.contains(r'http')]

# remove RT, @
df['text'] = df['text'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
df[df['text'].str.contains(r'RT[ ]?@')]
df['text'] = df['text'].str.replace(r'@[\S]+',r'')

#remove non-ascii words and characters
df['text'] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in df['text']]
df['text'] = df['text'].str.replace(r'_[\S]?',r'')

#remove &, < and >
df['text'] = df['text'].str.replace(r'&amp;?',r'and')
df['text'] = df['text'].str.replace(r'&lt;',r'<')
df['text'] = df['text'].str.replace(r'&gt;',r'>')

# remove extra space
df['text'] = df['text'].str.replace(r'[ ]{2, }',r' ')

# insert space between punctuation marks
df['text'] = df['text'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
df['text'] = df['text'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')


# lower case and strip white spaces at both ends
df['text'] = df['text'].str.lower()

# replace common abbreviations
df['text'] = df['text'].str.replace(r' u ', r' you ')
df['text'] = df['text'].str.replace(r' r ', r' are ')
df['text'] = df['text'].str.replace(r' some1 ', r' someone ')
df['text'] = df['text'].str.replace(r' yr ', r' year ')
df['text'] = df['text'].str.replace(r' yrs ', r' years ')
df['text'] = df['text'].str.replace(r' hr ', r' hour ')
df['text'] = df['text'].str.replace(r' hrs ', r' hours ')
df['text'] = df['text'].str.replace(r' min ', r' minute ')
df['text'] = df['text'].str.replace(r' mins ', r' minutes ')
df['text'] = df['text'].str.replace(r' sec ', r' second ')
df['text'] = df['text'].str.replace(r' secs ', r' seconds ')
df['text'] = df['text'].str.replace(r' pls ', r' please ')
df['text'] = df['text'].str.replace(r' plz ', r' please ')
df['text'] = df['text'].str.replace(r' 2morow ', r' tomorrow ')
df['text'] = df['text'].str.replace(r' 2morrow ', r' tomorrow ')
df['text'] = df['text'].str.replace(r' 2day ', r' today ')
df['text'] = df['text'].str.replace(r' 4got ', r' forgot ')
df['text'] = df['text'].str.replace(r' 4get ', r' forget ')
# df['text'] = df['text'].str.replace(r'4gotten', r'forgotten')

df['text'] = df['text'].str.strip()


In [ ]:
#Shuffle values since they were ordered by target value
df = df.sample(frac = 1)
df.head()

,target,id,date,user,text
1172441,1,1980629935,Sun May 31 07:15:50 PDT 2009,youngxr6,cant you go to hotel and watch it or you still...
184190,0,1967616783,Fri May 29 19:54:32 PDT 2009,kiwifoto,sorry to hear you guys won ' t be able to make...
794126,0,2326871128,Thu Jun 25 07:37:16 PDT 2009,ambersturgis,- my fingers were super swollen this morning !...
592615,0,2218010822,Wed Jun 17 21:44:24 PDT 2009,prateekgupta,yeah i know ... been a bit busy lately ... bu...
844255,1,1564023725,Mon Apr 20 01:23:07 PDT 2009,gift_of_game,i brought the sun back


In [ ]:
#Convert date strings to datetime format and seperate into new features
df['time'] =  pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S PDT %Y').dt.time
df['weekday'] =  pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S PDT %Y').dt.weekday
df['month'] =  pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S PDT %Y').dt.month
df['day'] =  pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S PDT %Y').dt.day

df['time'] =  pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S PDT %Y').dt.strftime('%H:%M:%S')

df.head()

,target,id,date,user,text,time,weekday,month,day
1172441,1,1980629935,Sun May 31 07:15:50 PDT 2009,youngxr6,cant you go to hotel and watch it or you still...,07:15:50,6,5,31
184190,0,1967616783,Fri May 29 19:54:32 PDT 2009,kiwifoto,sorry to hear you guys won ' t be able to make...,19:54:32,4,5,29
794126,0,2326871128,Thu Jun 25 07:37:16 PDT 2009,ambersturgis,- my fingers were super swollen this morning !...,07:37:16,3,6,25
592615,0,2218010822,Wed Jun 17 21:44:24 PDT 2009,prateekgupta,yeah i know ... been a bit busy lately ... bu...,21:44:24,2,6,17
844255,1,1564023725,Mon Apr 20 01:23:07 PDT 2009,gift_of_game,i brought the sun back,01:23:07,0,4,20


In [ ]:
df.drop('date', axis=1)
df.head()

,target,id,date,user,text,time,weekday,month,day
1172441,1,1980629935,Sun May 31 07:15:50 PDT 2009,youngxr6,cant you go to hotel and watch it or you still...,07:15:50,6,5,31
184190,0,1967616783,Fri May 29 19:54:32 PDT 2009,kiwifoto,sorry to hear you guys won ' t be able to make...,19:54:32,4,5,29
794126,0,2326871128,Thu Jun 25 07:37:16 PDT 2009,ambersturgis,- my fingers were super swollen this morning !...,07:37:16,3,6,25
592615,0,2218010822,Wed Jun 17 21:44:24 PDT 2009,prateekgupta,yeah i know ... been a bit busy lately ... bu...,21:44:24,2,6,17
844255,1,1564023725,Mon Apr 20 01:23:07 PDT 2009,gift_of_game,i brought the sun back,01:23:07,0,4,20


In [ ]:
#Convert Dataframes into proper format for tensorflow datasets
df_tfFormat = df.drop(['id', 'date', 'user', 'time', 'weekday', 'month', 'day'], axis=1)

df_tfFormat = df_tfFormat.rename(columns={"text": "DATA_COLUMN", "target": "LABEL_COLUMN"})

columns_titles = ["DATA_COLUMN","LABEL_COLUMN"]
df_tfFormat=df_tfFormat.reindex(columns=columns_titles)

df_tfFormat.head()

,DATA_COLUMN,LABEL_COLUMN
1172441,cant you go to hotel and watch it or you still...,1
184190,sorry to hear you guys won ' t be able to make...,0
794126,- my fingers were super swollen this morning !...,0
592615,yeah i know ... been a bit busy lately ... bu...,0
844255,i brought the sun back,1


In [ ]:
#Create train test split of our dataset now that it is formatted how we want
train_whole, test = train_test_split(df_tfFormat, test_size=0.3, random_state=42, shuffle=True)

train, validation = train_test_split(train_whole, test_size=0.2, random_state=42, shuffle=True)


In [ ]:
#Create tensorflow datasets for our train and test datasets we created
def convert_data_to_examples(train, validation, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = validation.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  test_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples, test_InputExamples

  train_InputExamples, validation_InputExamples, test_InputExamples = convert_data_to_examples(train, validation,
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'




In [ ]:
#Create Tensorflow datasets
train_InputExamples, validation_InputExamples, test_InputExamples = convert_data_to_examples(train, validation, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

test_data = convert_examples_to_tf_dataset(list(test_InputExamples), tokenizer)
test_data = test_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#Fit BERT model using Tensorflow datasets
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=1, validation_data=validation_data)

56000/56000 [==============================] - 27874s 497ms/step - loss: 0.3063 - accuracy: 0.8689 - val_loss: 0.3328 - val_accuracy: 0.8661


In [ ]:
loss, accuracy = model.evaluate(test_data)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

15000/15000 [==============================] - 2544s 170ms/step - loss: 0.3368 - accuracy: 0.8645
Loss: 0.3367745876312256
Accuracy: 0.8644999861717224
